In [ ]:
!pip install pycuda

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import time

In [ ]:
def load_ppm(file_path):
    with open(file_path, 'rb') as f:
        ppm_type = f.readline().strip()
        if ppm_type == b'P6':
            is_color = True
            print("Loaded color image")
        elif ppm_type == b'P5':
            is_color = False
            print("Loaded black-white image")
        else:
            raise ValueError("Not supported file format")

        while True:
            line = f.readline()
            if line[0] != 35:
                break

        width, height = map(int, line.strip().split())
        max_val = int(f.readline().strip())
        if max_val != 255:
            raise ValueError("Not supported max value: {}".format(max_val))

        if is_color:
            data = np.fromfile(f, dtype=np.uint8).reshape((height, width, 3))
        else:
            data = np.fromfile(f, dtype=np.uint8).reshape((height, width, 1))

    return data, is_color

In [ ]:
def save_ppm(file_path, data, is_color):
    height, width = data.shape[:2]

    ppm_type = b'P6' if is_color else b'P5'
    header = f"{ppm_type.decode()}\n{width} {height}\n255\n".encode('ascii')

    with open(file_path, 'wb') as f:
        f.write(header)

        if is_color:
            if data.ndim == 2 or data.shape[2] != 3:
                raise ValueError("Waited 3 channels")
            data = data.astype(np.uint8)
        else:
            data = data.astype(np.uint8).squeeze()

        data.tofile(f)

In [ ]:
def cpu_max_filter(image_data):
    height, width, channels = image_data.shape
    output = np.zeros_like(image_data, dtype=np.uint8)
    for c in range(channels):
      for i in range(0, height):
          for j in range(0, width):
              start_i = max(0, i - 1)
              end_i = min(height, i + 2)
              start_j = max(0, j - 1)
              end_j = min(width, j + 2)

              region = image_data[start_i:end_i, start_j:end_j, c]

              output[i][j][c] = max(region.flatten())

    return output

In [ ]:
kernel = """
__global__ void max_filter_shared(unsigned char* input_image, unsigned char* output_image, int width, int height, int channels)
{
    int yInInput = blockIdx.y * blockDim.y + threadIdx.y;
    int xInInput = blockIdx.x * blockDim.x + threadIdx.x;
    int z = threadIdx.z;


    __shared__ unsigned char sharedmem[(16 + 2)][(16 + 2)][3];


    bool is_x_left = (threadIdx.x == 0), is_x_right = (threadIdx.x == 16 - 1 || xInInput == width - 3);
    bool is_y_top = (threadIdx.y == 0), is_y_bottom = (threadIdx.y == 16 - 1 || yInInput == height - 3);


    if (yInInput + 2 < height && xInInput + 2 < width) {

        sharedmem[threadIdx.x + 1][threadIdx.y + 1][z] = input_image[((yInInput + 1) * width + xInInput + 1) * channels + z];

        if (is_x_left) {
            sharedmem[threadIdx.x][threadIdx.y + 1][z] = input_image[((yInInput + 1) * width + xInInput) * channels + z];
            if (is_y_bottom)
                sharedmem[threadIdx.x][threadIdx.y + 2][z] = input_image[((yInInput + 2) * width + xInInput) * channels + z];
            else if (is_y_top)
                sharedmem[threadIdx.x][threadIdx.y][z] = input_image[(yInInput * width + xInInput) * channels + z];
        }
        else if (is_x_right) {
            sharedmem[threadIdx.x + 2][threadIdx.y + 1][z] = input_image[((yInInput + 1) * width + xInInput + 2) * channels + z];
            if (is_y_bottom)
                sharedmem[threadIdx.x + 2][threadIdx.y + 2][z] = input_image[((yInInput + 2) * width + xInInput + 2) * channels + z];
            else if (is_y_top)
                sharedmem[threadIdx.x + 2][threadIdx.y][z] = input_image[(yInInput * width + xInInput + 2) * channels + z];
        }

        if (is_y_top)
            sharedmem[threadIdx.x + 1][threadIdx.y][z] = input_image[(yInInput * width + xInInput + 1) * channels + z];
        else if (is_y_bottom)
            sharedmem[threadIdx.x + 1][threadIdx.y + 2][z] = input_image[((yInInput + 2) * width + xInInput + 1) * channels + z];

        __syncthreads();

        unsigned char a[9] = { sharedmem[threadIdx.x][threadIdx.y][z], sharedmem[threadIdx.x + 1][threadIdx.y][z], sharedmem[threadIdx.x + 2][threadIdx.y][z],
                        sharedmem[threadIdx.x][threadIdx.y + 1][z], sharedmem[threadIdx.x + 1][threadIdx.y + 1][z], sharedmem[threadIdx.x + 2][threadIdx.y + 1][z],
                        sharedmem[threadIdx.x][threadIdx.y + 2][z], sharedmem[threadIdx.x + 1][threadIdx.y + 2][z], sharedmem[threadIdx.x + 2][threadIdx.y + 2][z] };


        unsigned char max_value = a[0];
        for (int i = 1; i < 9; i++) {
            max_value = max(max_value, a[i]);
        }

        output_image[(yInInput * width + xInInput) * channels + z] = max_value;
    }
}
"""

mod = SourceModule(kernel)
max_filter = mod.get_function("max_filter_shared")

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


In [ ]:
kernel = """
__global__ void max_filter_shared(unsigned char* input_image, unsigned char* output_image, int width, int height, int channels) {
    // Индексы текущего потока
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int c = threadIdx.z;

    // Размеры локальных индексов для shared memory (с учетом рамки для окна 3x3)
    int local_x = threadIdx.x + 1;
    int local_y = threadIdx.y + 1;

    // Shared memory для блока с учётом границ
    __shared__ unsigned char shared_mem[16 + 2][16 + 2][3];

    // Инициализируем shared memory нулями для избежания ошибок вне изображения
    shared_mem[local_x][local_y][c] = 0;
    if (threadIdx.x == 0 || threadIdx.y == 0 || threadIdx.x == blockDim.x - 1 || threadIdx.y == blockDim.y - 1) {
        shared_mem[local_x - 1][local_y - 1][c] = 0;
        shared_mem[local_x + 1][local_y + 1][c] = 0;
    }

    // Загружаем центральные пиксели блока в shared memory
    if (x < width && y < height) {
        shared_mem[local_x][local_y][c] = input_image[(y * width + x) * channels + c];
    }

    // Заполняем границы shared memory, если поток находится на краю блока
    if (threadIdx.x == 0 && x > 0) {
        shared_mem[local_x - 1][local_y][c] = input_image[(y * width + (x - 1)) * channels + c];  // Левый сосед
    }
    if (threadIdx.x == blockDim.x - 1 && x < width - 1) {
        shared_mem[local_x + 1][local_y][c] = input_image[(y * width + (x + 1)) * channels + c];  // Правый сосед
    }
    if (threadIdx.y == 0 && y > 0) {
        shared_mem[local_x][local_y - 1][c] = input_image[((y - 1) * width + x) * channels + c];  // Верхний сосед
    }
    if (threadIdx.y == blockDim.y - 1 && y < height - 1) {
        shared_mem[local_x][local_y + 1][c] = input_image[((y + 1) * width + x) * channels + c];  // Нижний сосед
    }

    // Угловые соседи
    if (threadIdx.x == 0 && threadIdx.y == 0 && x > 0 && y > 0) {
        shared_mem[local_x - 1][local_y - 1][c] = input_image[((y - 1) * width + (x - 1)) * channels + c];  // Левый верхний
    }
    if (threadIdx.x == blockDim.x - 1 && threadIdx.y == 0 && x < width - 1 && y > 0) {
        shared_mem[local_x + 1][local_y - 1][c] = input_image[((y - 1) * width + (x + 1)) * channels + c];  // Правый верхний
    }
    if (threadIdx.x == 0 && threadIdx.y == blockDim.y - 1 && x > 0 && y < height - 1) {
        shared_mem[local_x - 1][local_y + 1][c] = input_image[((y + 1) * width + (x - 1)) * channels + c];  // Левый нижний
    }
    if (threadIdx.x == blockDim.x - 1 && threadIdx.y == blockDim.y - 1 && x < width - 1 && y < height - 1) {
        shared_mem[local_x + 1][local_y + 1][c] = input_image[((y + 1) * width + (x + 1)) * channels + c];  // Правый нижний
    }

    // Ждём завершения загрузки данных в shared memory
    __syncthreads();

    // Проверяем, что поток находится внутри границ изображения
    if (x < width && y < height) {
        int max_pixel = 0;

        // Обрабатываем границы изображения, чтобы не выйти за их пределы
        int start_x = max(1, local_x - 1);
        int end_x = min(16 + 1, local_x + 1);
        int start_y = max(1, local_y - 1);
        int end_y = min(16 + 1, local_y + 1);

        // Поиск максимального значения в окне 3x3
        for (int i = start_x; i <= end_x; i++) {
            for (int j = start_y; j <= end_y; j++) {
                max_pixel = max(max_pixel, (int)shared_mem[i][j][c]);
            }
        }

        // Ждём завершения обработки данных
        __syncthreads();

        // Записываем результат в глобальную память
        output_image[(y * width + x) * channels + c] = (unsigned char)max_pixel;
    }
}
"""
mod = SourceModule(kernel)
max_filter = mod.get_function("max_filter_shared")


In [ ]:
kernel = """
__global__ void max_filter(unsigned char* input_image, unsigned char* output_image, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int c = threadIdx.z;

    if (x < width && y < height && c < channels) {

        int max_pixel = 0;

        int start_i = max(0, x - 1);
        int end_i = min(width, x + 2);
        int start_j = max(0, y - 1);
        int end_j = min(height, y + 2);

        for (int i = start_i; i < end_i; i++) {
            for (int j = start_j; j < end_j; j++) {
                int pixel = input_image[(j * width + i) * channels + c];
                if (max_pixel < pixel) {
                    max_pixel = pixel;
                }
            }
        }

        output_image[(y * width + x) * channels + c] = (unsigned char)max_pixel;
    }
}
"""
mod = SourceModule(kernel)
max_filter = mod.get_function("max_filter")

In [ ]:
def apply_max_filter(image_data):
    height, width, channels = image_data.shape
    image_data_flat = image_data.ravel()

    input_image_gpu = cuda.mem_alloc(image_data_flat.nbytes)
    output_image_gpu = cuda.mem_alloc(image_data_flat.nbytes)

    cuda.memcpy_htod(input_image_gpu, image_data_flat)

    block_size = (16, 16, channels)
    grid_size = (width // block_size[0] + 1, height // block_size[1] + 1, 1)

    start_event = cuda.Event()
    end_event = cuda.Event()
    start_event.record()

    max_filter(input_image_gpu, output_image_gpu, np.int32(width), np.int32(height), np.int32(channels), block=block_size, grid=grid_size)

    end_event.record()
    end_event.synchronize()
    gpu_time = start_event.time_till(end_event)

    output_image = np.empty_like(image_data_flat)
    cuda.memcpy_dtoh(output_image, output_image_gpu)
    output_image = output_image.reshape((height, width, channels))

    input_image_gpu.free()
    output_image_gpu.free()

    return output_image, gpu_time

In [ ]:
image_path = "1695128011374.pgm"
image_data, is_color = load_ppm(image_path)

start_time = time.perf_counter()

cpu_res = cpu_max_filter(image_data)

cpu_time = (time.perf_counter() - start_time) * 1000
print(f"CPU execution time is {cpu_time:.1f} ms")

# save_ppm("example3_cpu_res.ppm", cpu_res, is_color)

gpu_res, gpu_time = apply_max_filter(image_data)
print(f"GPU execution time is {gpu_time:.1f} ms")
print("Is equal:", np.array_equal(gpu_res, cpu_res))

# save_ppm("example3_gpu_res.ppm", gpu_res, is_color)


Loaded black-white image
CPU execution time is 39812.2 ms
GPU execution time is 1.7 ms
Is equal: True
